<a href="https://colab.research.google.com/github/baileysmoko/Fabric/blob/main/Dune_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade dune-client duckdb pandas pyarrow python-dotenv requests


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sou

In [ ]:
import os
os.environ['DUNE_API_KEY'] = "xxx"


In [ ]:
import requests, io, pandas as pd, os

query_id = 5752163
API_KEY = os.environ['DUNE_API_KEY']
headers = {"x-dune-api-key": API_KEY}

out_dir = "/content/dune_parquet_shards"
os.makedirs(out_dir, exist_ok=True)

limit = 100_000
offset = 0
chunk = 0

while True:
    url = f"https://api.dune.com/api/v1/query/{query_id}/results/csv"
    params = {"limit": limit, "offset": offset}
    r = requests.get(url, headers=headers, params=params, timeout=600)
    r.raise_for_status()

    df = pd.read_csv(io.StringIO(r.text))
    if df.empty:
        print("All data downloaded!")
        break

    out_path = f"{out_dir}/chunk_{chunk:04d}.parquet"
    df.to_parquet(out_path, index=False, compression="snappy")
    print(f"Saved {len(df)} rows → {out_path}")

    offset += len(df)
    chunk += 1

    # safety stop for testing
    # if chunk >= 5:  # remove this line once you’re confident it works
    #     break


Saved 32000 rows → /content/dune_parquet_shards/chunk_0000.parquet
Saved 32000 rows → /content/dune_parquet_shards/chunk_0001.parquet
Saved 32000 rows → /content/dune_parquet_shards/chunk_0002.parquet
Saved 32000 rows → /content/dune_parquet_shards/chunk_0003.parquet
Saved 32000 rows → /content/dune_parquet_shards/chunk_0004.parquet
Saved 32000 rows → /content/dune_parquet_shards/chunk_0005.parquet
Saved 32000 rows → /content/dune_parquet_shards/chunk_0006.parquet
Saved 32000 rows → /content/dune_parquet_shards/chunk_0007.parquet
Saved 32000 rows → /content/dune_parquet_shards/chunk_0008.parquet
Saved 32000 rows → /content/dune_parquet_shards/chunk_0009.parquet
Saved 32000 rows → /content/dune_parquet_shards/chunk_0010.parquet
Saved 32000 rows → /content/dune_parquet_shards/chunk_0011.parquet
Saved 32000 rows → /content/dune_parquet_shards/chunk_0012.parquet
Saved 32000 rows → /content/dune_parquet_shards/chunk_0013.parquet
Saved 32000 rows → /content/dune_parquet_shards/chunk_0014.par

In [ ]:
import duckdb

con = duckdb.connect()
con.execute("""
    CREATE VIEW top100 AS
    SELECT * FROM read_parquet('/content/dune_parquet_shards/*.parquet')
""")

# Count rows
print(con.execute("SELECT count(*) FROM top100").fetchall())

# See a preview
print(con.execute("SELECT * FROM top100 LIMIT 5").fetchdf())



[(160000,)]
                          date symbol  \
0  2010-07-31 00:00:00.000 UTC    BTC   
1  2010-07-31 00:00:00.000 UTC    BTC   
2  2010-07-31 01:00:00.000 UTC    BTC   
3  2010-07-31 01:00:00.000 UTC    BTC   
4  2010-07-31 02:00:00.000 UTC    BTC   

                                    contract_address   price volume  
0         0x0000000000000000000000000000000000000000  0.0627  <nil>  
1  0x8268e9a9a1444c2ba5c77a51936856b072e43fefcff5...  0.0627  <nil>  
2         0x0000000000000000000000000000000000000000  0.0627  <nil>  
3  0x8268e9a9a1444c2ba5c77a51936856b072e43fefcff5...  0.0627  <nil>  
4         0x0000000000000000000000000000000000000000  0.0627  <nil>  


In [ ]:
total_rows = con.execute("SELECT count(*) FROM top100").fetchone()[0]
print(f"Total rows: {total_rows:,}")


Total rows: 160,000
